In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir -p datasets/flores101
!mkdir -p datasets/in22
!mkdir -p models
!mkdir -p translations


In [4]:
!mkdir -p '/content/drive/My Drive/IndicTrans2/datasets'
!mkdir -p '/content/drive/My Drive/IndicTrans2/models'
!mkdir -p '/content/drive/My Drive/IndicTrans2/translations'


In [2]:
!git clone https://github.com/facebookresearch/flores.git


fatal: destination path 'flores' already exists and is not an empty directory.


In [3]:
%cd flores
!bash scripts/download_flores101_dataset.sh
%cd ..


/content/flores
bash: scripts/download_flores101_dataset.sh: No such file or directory
/content


In [4]:
!wget -P datasets/in22/ https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/translation/indicat/IN22/IN22-Gen.zip
!wget -P datasets/in22/ https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/translation/indicat/IN22/IN22-Conv.zip

!unzip datasets/in22/IN22-Gen.zip -d datasets/in22/
!unzip datasets/in22/IN22-Conv.zip -d datasets/in22/


--2024-11-28 10:50:05--  https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/translation/indicat/IN22/IN22-Gen.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.12.27, 2607:f8b0:4025:815::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-28 10:50:06 ERROR 404: Not Found.

--2024-11-28 10:50:06--  https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/translation/indicat/IN22/IN22-Conv.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.12.27, 2607:f8b0:4025:815::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-28 10:50:07 ERROR 404: Not Found.

unzip:  cannot find or open datasets/in22/IN22-Gen.zip, datasets/in22/IN22-Gen.zip.zip or datasets/in22/IN22-Gen.zip.ZI

datasets/
├── flores101/
│   ├── dev/
│   ├── devtest/
├── in22/
    ├── IN22-Gen/
    ├── IN22-Conv/


In [5]:
# Install necessary packages
!pip install sacremoses sentencepiece
!pip install sacrebleu
!pip install transformers==4.24.0
!pip install fairseq==0.12.2
!pip install omegaconf
!pip install pandas numpy


  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached hydra_core-1.0.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl.metadata (3.0 kB)
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl (from fairseq==0.12.2) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>=5.1.*)
            ~~~~~~^
Please use pip<24.1 if you need to use this version.
  Using cached omegaconf-2.0.5-py3-none-any.whl.metadata (3.0 kB)
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/e5/f6/043b6d255dd6fbf2025110cea35b87f4c5100a181681d8eab496269f0d5b/omegaconf-2.0.5-py3-none-any.whl (from fairseq==0.12.2) has invalid metadata: 

In [6]:
!git clone https://github.com/AI4Bharat/IndicTrans2.git
%cd IndicTrans2


fatal: destination path 'IndicTrans2' already exists and is not an empty directory.
/content/IndicTrans2


In [7]:
pip install datasets

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [8]:
#!huggingface-cli login

In [9]:
!pip install transformers datasets joblib openai tqdm


In [10]:
'''import os
import json
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from joblib import Parallel, delayed
from openai import APIStatusError, OpenAI

# Constants and Language Mappings
_TEMPERATURE = 0
_MAX_TOKENS = 16384
_TOP_P = 1
_TOP_K = 64
_FREQUENCY_PENALTY = 0
_PRESENCE_PENALTY = 0

_FLORES_MAP = {
    "Assamese": "asm_Beng",
    "Bengali": "ben_Beng",
    "Gujarati": "guj_Gujr",
    "Hindi": "hin_Deva",
    "Kannada": "kan_Knda",
    "Malayalam": "mal_Mlym",
    "Marathi": "mar_Deva",
    "Nepali": "npi_Deva",
    "Odia": "ory_Orya",
    "Punjabi": "pan_Guru",
    "Sanskrit": "san_Deva",
    "Tamil": "tam_Taml",
    "Telugu": "tel_Telu",
    "Urdu": "urd_Arab",
    "Bodo": "brx_Deva",
    "Dogri": "doi_Deva",
    "Konkani": "gom_Deva",
    "Kashmiri": "kas_Arab",
    "Maithili": "mai_Deva",
    "Manipuri": "mni_Mtei",
    "Santali": "sat_Olck",
    "Sindhi": "snd_Deva"
}

_LANG_MAP = {
    "asm": "Assamese",
    "ben": "Bengali",
    "guj": "Gujarati",
    "hin": "Hindi",
    "kan": "Kannada",
    "mal": "Malayalam",
    "mar": "Marathi",
    "nep": "Nepali",
    "ory": "Odia",
    "pan": "Punjabi",
    "san": "Sanskrit",
    "tam": "Tamil",
    "tel": "Telugu",
    "urd": "Urdu",
    "brx": "Bodo",
    "doi": "Dogri",
    "gom": "Konkani",
    "kas": "Kashmiri",
    "mai": "Maithili",
    "mni": "Manipuri",
    "sat": "Santali",
    "snd": "Sindhi"
}

# Define the translation function
def translate_batch(system_prompt, task_prompt, texts, lang):
    """
    Translate a batch of texts using the OpenAI API.
    """
    # Initialize the OpenAI client inside the function to avoid pickling issues
    llama = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key="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"  # Replace with your actual API key
    )

    # Construct the input prompt with numbered sentences
    _TRANSLATE_MODEL_INPUT = f"{task_prompt}\n\n"
    for idx, text in enumerate(texts, 1):
        _TRANSLATE_MODEL_INPUT += f"{idx}. {text}\n"
    _TRANSLATE_MODEL_INPUT += "\nTranslations:\n"

    try:
        response = llama.chat.completions.create(
            model="meta/llama-3.1-405b-instruct",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": _TRANSLATE_MODEL_INPUT}
            ],
            temperature=_TEMPERATURE,
            max_tokens=_MAX_TOKENS,
            top_p=_TOP_P,
            frequency_penalty=_FREQUENCY_PENALTY,
            presence_penalty=_PRESENCE_PENALTY
        )
        translations_text = response.choices[0].message.content.strip()

        # Parse the translations into a list
        translations = []
        for line in translations_text.split('\n'):
            if line.strip():
                # Assuming format: "1. [translation]"
                parts = line.strip().split('.', 1)
                if len(parts) == 2:
                    translation = parts[1].strip()
                    translations.append(translation)

        # Ensure that we have the same number of translations as inputs
        if len(translations) != len(texts):
            print(f"Warning: Expected {len(texts)} translations, but got {len(translations)}.")
            # Fill missing translations with empty strings
            translations += [""] * (len(texts) - len(translations))

        return translations
    except APIStatusError as e:
        if e.status_code in {402, 401}:
            print("Issue with the API key or payment, skipping these translations...")
            return [""] * len(texts)
        else:
            print(f"APIStatusError: {e}")
            return [""] * len(texts)
    except Exception as e:
        print(f"Generation error encountered: {e}")
        return [""] * len(texts)

# Define the batch processing function
def process_translation_batch(batch):
    """
    Process a single batch of translations.
    Each batch contains up to 10 rows.
    """
    batch_index, rows, lang = batch
    target_lang = _LANG_MAP.get(lang, "Unknown")

    _MT_SYSTEM_PROMPT = f"You are a machine translation model. Please help in translating the following text to {target_lang}."
    _MT_TASK_PROMPT = (
        f"Translate the following English text to {target_lang}. Follow these guidelines:\n\n"
        "1. **Accuracy:** Translate the text as accurately as possible, ensuring no information is missed.\n"
        "2. **Tone and Naturalness:** Maintain the original tone and ensure the translation sounds natural.\n"
        "3. **Meaningful Restructuring:** Do not translate word-for-word. Restructure sentences if needed.\n"
        "4. **Do not Translate the numericals.\n"
        "5. **Do not provide any explanations.\n\n"
        "Ensure that the translated text is clear, coherent, and grammatically correct.\n\n"
    )

    # Extract English sentences from the batch
    texts = [row['sentence_eng_Latn'] for row in rows]

    # Perform batch translation
    translated_texts = translate_batch(_MT_SYSTEM_PROMPT, _MT_TASK_PROMPT, texts, lang)

    # Define the output directory (ensure it's mounted on Google Drive)
    output_dir = f"/content/drive/MyDrive/{lang}"
    os.makedirs(output_dir, exist_ok=True)

    # Save each translation in its respective JSONL file
    for j, translated_text in enumerate(translated_texts):
        row = rows[j]
        output_data = {
            'source_lang': 'eng_Latn',
            'target_lang': _FLORES_MAP.get(_LANG_MAP.get(lang, ""), "unknown"),
            'meta': {
                'model_name': 'Meta-Llama-3.1-405B-Instruct-FP32',
                'temperature': _TEMPERATURE,
                'max_tokens': _MAX_TOKENS,
                'top_p': _TOP_P,
                'frequency_penalty': _FREQUENCY_PENALTY,
                'presence_penalty': _PRESENCE_PENALTY
            },
            'Input Prompt': row['sentence_eng_Latn'],
            'translated_text': translated_text,
            'id': int(row['id']),
        }
        # Optional: Remove or comment out to reduce output clutter
        # print(output_data)

        # Save to JSONL
        with open(f"{output_dir}/{lang}_translations_{batch_index + j}.jsonl", 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False)

# Function to create batches
def create_batches(df, batch_size=10):
    """
    Split the DataFrame into batches of specified size.
    """
    batches = []
    total = len(df)
    for i in range(0, total, batch_size):
        batch_rows = df.iloc[i:i+batch_size].to_dict(orient='records')
        batches.append( (i, batch_rows, lang) )
    return batches

# Main Execution
if __name__ == "__main__":
    # Load Google Drive (if not already mounted)
    from google.colab import drive
    drive.mount('/content/drive')

    # Load the dataset for English to Marathi translation
    in22_dataset = load_dataset("ai4bharat/IN22-Conv", "eng_Latn-tel_Telu")
    df = pd.DataFrame(in22_dataset["conv"])
    lang = "tel"  # Set your target language code herea
    # Create batches of 10 sentences each
    batch_size = 10
    batches = create_batches(df, batch_size=batch_size)

    # Set the number of parallel jobs (adjust based on CPU and API rate limits)
    num_jobs = 5  # Adjust to 5 or 10 as per your requirement

    # Run translations in parallel using threading backend to avoid pickling issues
    Parallel(n_jobs=num_jobs, backend="threading")(
        delayed(process_translation_batch)(batch) for batch in tqdm(batches)
    )
'''

'import os\nimport json\nimport pandas as pd\nfrom tqdm import tqdm\nfrom datasets import load_dataset\nfrom joblib import Parallel, delayed\nfrom openai import APIStatusError, OpenAI\n\n# Constants and Language Mappings\n_TEMPERATURE = 0\n_MAX_TOKENS = 16384\n_TOP_P = 1\n_TOP_K = 64\n_FREQUENCY_PENALTY = 0\n_PRESENCE_PENALTY = 0\n\n_FLORES_MAP = {\n    "Assamese": "asm_Beng",\n    "Bengali": "ben_Beng",\n    "Gujarati": "guj_Gujr",\n    "Hindi": "hin_Deva",\n    "Kannada": "kan_Knda",\n    "Malayalam": "mal_Mlym",\n    "Marathi": "mar_Deva",\n    "Nepali": "npi_Deva",\n    "Odia": "ory_Orya",\n    "Punjabi": "pan_Guru",\n    "Sanskrit": "san_Deva",\n    "Tamil": "tam_Taml",\n    "Telugu": "tel_Telu",\n    "Urdu": "urd_Arab",\n    "Bodo": "brx_Deva",\n    "Dogri": "doi_Deva",\n    "Konkani": "gom_Deva",\n    "Kashmiri": "kas_Arab",\n    "Maithili": "mai_Deva",\n    "Manipuri": "mni_Mtei",\n    "Santali": "sat_Olck",\n    "Sindhi": "snd_Deva"\n}\n\n_LANG_MAP = {\n    "asm": "Assamese",\

In [ ]:
import os
import json
import argparse
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer
from joblib import Parallel, delayed
from openai import APIStatusError, OpenAI

'''parser = argparse.ArgumentParser(description="Run the LLM translation for Frames dataset")
parser.add_argument("--lang", type=str, required=True, help="Target language for translation")
args = parser.parse_args()'''

_TEMPERATURE = 0
_MAX_TOKENS = 16384
_TOP_P = 1
_TOP_K = 64
_FREQUENCY_PENALTY = 0
_PRESENCE_PENALTY = 0

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-405B",use_auth_token =True)

# Language mappings
_FLORES_MAP = {
    "Assamese": "asm_Beng",
    "Bengali": "ben_Beng",
    "Gujarati": "guj_Gujr",
    "Hindi": "hin_Deva",
    "Kannada": "kan_Knda",
    "Malayalam": "mal_Mlym",
    "Marathi": "mar_Deva",
    "Nepali": "npi_Deva",
    "Odia": "ory_Orya",
    "Punjabi": "pan_Guru",
    "Sanskrit": "san_Deva",
    "Tamil": "tam_Taml",
    "Telugu": "tel_Telu",
    "Urdu": "urd_Arab",
    "Bodo" : "brx_Deva",
    "Dogri":"doi_Deva",
    "Konkani":"gom_Deva",
    "Kashmiri":"kas_Arab",
    "Maithili":"mai_Deva",
    "Nepali":'npi_Deva',
    "Manipuri": 'mni_Mtei',
    "Santali":'sat_Olck',
    "Sindhi":'snd_Deva'
}

_LANG_MAP = {
    "asm": "Assamese",
    "ben": "Bengali",
    "guj": "Gujarati",
    "hin": "Hindi",
    "kan": "Kannada",
    "mal": "Malayalam",
    "mar": "Marathi",
    "nep": "Nepali",
    "ory": "Odia",
    "pan": "Punjabi",
    "san": "Sanskrit",
    "tam": "Tamil",
    "tel": "Telugu",
    "urd": "Urdu",
    "brx":"Bodo",
    "doi":"Dogri",
    "gom":"Konkani",
    "kas":"Kashmiri",
    "mai":"Maithili",
    "npi":"Nepali",
    "mni":"Manipuri",
    "sat":"Santali",
    "snd":"Sindhi"


}

# Load Frames dataset


def translate_text(llama, system_prompt, task_prompt, text):
    """Helper function to make a translation API call for a single text input."""
    _TRANSLATE_MODEL_INPUT = (
        f"{task_prompt}"
        f"Input Text: {text}\n"
        f"Translated Text: \n"
    )
    # print(_TRANSLATE_MODEL_INPUT)
    try:
        response = llama.chat.completions.create(
            #model="meta-llama/Meta-Llama-3.1-405B-Instruct-FP8",
            model="meta/llama-3.1-405b-instruct",
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": _TRANSLATE_MODEL_INPUT}],
            temperature=0,
            max_tokens=4096,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        # print(response.choices[0].message.content)
        return response.choices[0].message.content
    except APIStatusError as e:
        if e.status_code in {402, 401}:
            print("Issue with the key, skipping...")
            return ""
        return ""
    except Exception as e:
        print("Generation error encountered:", e)
        return ""

def process_translation(i, row):
    llama = OpenAI(
        base_url = "https://integrate.api.nvidia.com/v1",
        api_key = "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"
    )
    target_lang = _LANG_MAP[lang]

    _MT_SYSTEM_PROMPT = f"You are a machine translation model. Please help in translating the following text to {target_lang}."
    _MT_TASK_PROMPT = (
        f"Translate the following English text to {target_lang}. Follow these guidelines:\n\n"
        "1. **Accuracy:** Translate the text as accurately as possible, ensuring no information is missed.\n"
        "2. **Tone and Naturalness:** Maintain the original tone and ensure the translation sounds natural.\n"
        "3. **Meaningful Restructuring:** Do not translate word-for-word. Restructure sentences if needed.\n"
        "4. **Do not Translate the numericals\n"
        "5. **Do not provide any Explnations.\n\n"
        "Ensure that the translated text is clear, coherent, and grammatically correct.\n\n"
    )

    # Translate Prompt and Answer
    translated_prompt = translate_text(llama, _MT_SYSTEM_PROMPT, _MT_TASK_PROMPT, row['sentence_eng_Latn'])
   # translated_answer = translate_text(llama, _MT_SYSTEM_PROMPT, _MT_TASK_PROMPT, row['Answer'])
    output_dir = f"/content/drive/MyDrive/{lang}"
    #output_dir = f"./in22-translations/{lang}"
    print(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    output_data = {
        'source_lang': 'eng_Latn',
        'target_lang': _FLORES_MAP[_LANG_MAP[lang]],
        'meta': {
            'model_name': 'Meta-Llama-3.1-405B-Instruct-FP32',
            'temperature': _TEMPERATURE,
            'max_tokens': _MAX_TOKENS,
            'top_p': _TOP_P,
            'frequency_penalty': _FREQUENCY_PENALTY,
            'presence_penalty': _PRESENCE_PENALTY
        },
        'Input Prompt': row['sentence_eng_Latn'],
        'translated_text': translated_prompt,
        'id': int(row['id']),
    }
    print(output_data)

    with open(f"{output_dir}/{lang}_translations_"+str(i)+".jsonl", 'w') as f:
        json.dump(output_data, f, ensure_ascii=False)




in22_dataset = load_dataset("ai4bharat/IN22-Conv","eng_Latn-tam_Taml")
df = pd.DataFrame(in22_dataset["conv"])
lang = "tam"





for i in range(337,900):
    process_translation(i, df.iloc[i])
# Run translations and gather results
#Parallel(n_jobs=2)(delayed(process_translation)(i, row) for i, row in tqdm(df.head(3).iterrows()))

# Write all results to a single JSON file with shared metadata

## Run the script with the following command:
# python generate_rag.py --lang=asm
# python generate_rag.py --lang=ben
# python generate_rag.py --lang=guj
# python generate_rag.py --lang=hin
# python generate_rag.py --lang=kan
# python generate_rag.py --lang=mal
# python generate_rag.py --lang=mar
# python generate_rag.py --lang=nep
# python generate_rag.py --lang=ori
# python generate_rag.py --lang=pan
# python generate_rag.py --lang=san
# python generate_rag.py --lang=tam
# python generate_rag.py --lang=tel
# python generate_rag.py --lang=urd



/content/drive/MyDrive/tam
{'source_lang': 'eng_Latn', 'target_lang': 'tam_Taml', 'meta': {'model_name': 'Meta-Llama-3.1-405B-Instruct-FP32', 'temperature': 0, 'max_tokens': 16384, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0}, 'Input Prompt': 'The government has promised us clean water, but we are not getting it in our village.', 'translated_text': 'அரசாங்கம் எங்களுக்கு சுத்தமான நீர் கொடுக்கப்படும் என்று வாக்குறுதி அளித்தது, ஆனால் எங்கள் கிராமத்தில் அது கிடைக்கவில்லை.', 'id': 338}
/content/drive/MyDrive/tam
{'source_lang': 'eng_Latn', 'target_lang': 'tam_Taml', 'meta': {'model_name': 'Meta-Llama-3.1-405B-Instruct-FP32', 'temperature': 0, 'max_tokens': 16384, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0}, 'Input Prompt': 'No inspection happened in your village till now?', 'translated_text': 'உங்கள் கிராமத்தில் இதுவரை ஆய்வு நடந்ததா?', 'id': 339}
/content/drive/MyDrive/tam
{'source_lang': 'eng_Latn', 'target_lang': 'tam_Taml', 'meta': {'model_name': 'Meta-Llama-

In [ ]:
import os
import json
import pandas as pd
from datasets import load_dataset
import evaluate

def compute_bleu_score(lang_code):
    # Mapping from language codes to dataset configurations and field names
    lang_config_map = {
        "asm": {"config": "eng_Latn-asm_Beng", "field": "sentence_asm_Beng"},
        "ben": {"config": "eng_Latn-ben_Beng", "field": "sentence_ben_Beng"},
        "guj": {"config": "eng_Latn-guj_Gujr", "field": "sentence_guj_Gujr"},
        "hin": {"config": "eng_Latn-hin_Deva", "field": "sentence_hin_Deva"},
        "kan": {"config": "eng_Latn-kan_Knda", "field": "sentence_kan_Knda"},
        "mal": {"config": "eng_Latn-mal_Mlym", "field": "sentence_mal_Mlym"},
        "mar": {"config": "eng_Latn-mar_Deva", "field": "sentence_mar_Deva"},
        "npi": {"config": "eng_Latn-npi_Deva", "field": "sentence_npi_Deva"},
        "ory": {"config": "eng_Latn-ory_Orya", "field": "sentence_ory_Orya"},
        "pan": {"config": "eng_Latn-pan_Guru", "field": "sentence_pan_Guru"},
        "san": {"config": "eng_Latn-san_Deva", "field": "sentence_san_Deva"},
        "tam": {"config": "eng_Latn-tam_Taml", "field": "sentence_tam_Taml"},
        "tel": {"config": "eng_Latn-tel_Telu", "field": "sentence_tel_Telu"},
        "urd": {"config": "eng_Latn-urd_Arab", "field": "sentence_urd_Arab"},
        # Add other languages as needed
    }

    # Check if lang_code is supported
    if lang_code not in lang_config_map:
        print(f"Language code {lang_code} not supported.")
        return

    lang_config = lang_config_map[lang_code]['config']
    lang_field = lang_config_map[lang_code]['field']

    # Path to the directory containing the JSON files of generated translations
    output_dir = f"/path/to/translations/{lang_code}"  # Replace with your actual path

    # Read all JSON files in the output_dir
    translations = []
    json_files = [f for f in os.listdir(output_dir) if f.endswith('.jsonl')]

    for json_file in json_files:
        with open(os.path.join(output_dir, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)
            translation = data.get('translated_text', '').strip()
            idx = data.get('id')
            translations.append({'id': idx, 'translation': translation})

    # Sort translations based on 'id'
    translations = sorted(translations, key=lambda x: x['id'])
    generated_translations = [t['translation'] for t in translations]
    ids = [t['id'] for t in translations]

    # Load the reference translations
    dataset = load_dataset("ai4bharat/IN22-Conv", lang_config, trust_remote_code=True)
    df = pd.DataFrame(dataset["conv"])

    # Get references aligned with the IDs
    references_df = df[df['id'].isin(ids)].sort_values('id')
    references = references_df[lang_field].tolist()

    # Ensure that the number of generated translations matches the number of references
    if len(generated_translations) != len(references):
        print("Warning: Number of generated translations and references do not match.")
        # Optionally, adjust the lists
        min_len = min(len(generated_translations), len(references))
        generated_translations = generated_translations[:min_len]
        references = references[:min_len]

    # Prepare the references in the format expected by the BLEU metric
    references = [[ref] for ref in references]

    # Load the BLEU metric
    bleu = evaluate.load("bleu")

    # Compute BLEU score
    results = bleu.compute(predictions=generated_translations, references=references)

    print(f"BLEU score for {lang_code}: {results['bleu']*100:.2f}")

# Example usage
compute_bleu_score('tam')  # Replace 'tam' with the language code you want to evaluate


In [ ]:
'''import os
import json
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer
from joblib import Parallel, delayed
from openai import APIStatusError, OpenAI
import threading

# Constants and language mappings remain the same
_TEMPERATURE = 0
_MAX_TOKENS = 16384
_TOP_P = 1
_TOP_K = 64
_FREQUENCY_PENALTY = 0
_PRESENCE_PENALTY = 0

_FLORES_MAP = {
    "Assamese": "asm_Beng",
    "Bengali": "ben_Beng",
    "Gujarati": "guj_Gujr",
    "Hindi": "hin_Deva",
    "Kannada": "kan_Knda",
    "Malayalam": "mal_Mlym",
    "Marathi": "mar_Deva",
    "Nepali": "npi_Deva",
    "Odia": "ory_Orya",
    "Punjabi": "pan_Guru",
    "Sanskrit": "san_Deva",
    "Tamil": "tam_Taml",
    "Telugu": "tel_Telu",
    "Urdu": "urd_Arab"
}

_LANG_MAP = {
    "asm": "Assamese",
    "ben": "Bengali",
    "guj": "Gujarati",
    "hin": "Hindi",
    "kan": "Kannada",
    "mal": "Malayalam",
    "mar": "Marathi",
    "nep": "Nepali",
    "ori": "Odia",
    "pan": "Punjabi",
    "san": "Sanskrit",
    "tam": "Tamil",
    "tel": "Telugu",
    "urd": "Urdu"
}

# Initialize the API client globally
llama = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="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"  # Replace with your actual API key
)

# Translation function remains the same
def translate_text(llama_client, system_prompt, task_prompt, text):
    """Helper function to make a translation API call for a single text input."""
    _TRANSLATE_MODEL_INPUT = (
        f"{task_prompt}"
        f"Input Text: {text}\n"
        f"Translated Text: \n"
    )
    try:
        response = llama_client.chat.completions.create(
            model="meta/llama-3.1-405b-instruct",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": _TRANSLATE_MODEL_INPUT}
            ],
            temperature=0,
            max_tokens=4096,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content.strip()
    except APIStatusError as e:
        if e.status_code in {402, 401}:
            print("Issue with the API key, skipping...")
            return ""
        return ""
    except Exception as e:
        print("Generation error encountered:", e)
        return ""

# Modify process_translation to accept the llama client as a parameter
def process_translation(args):
    i, row, lang = args
    target_lang = _LANG_MAP[lang]

    _MT_SYSTEM_PROMPT = f"You are a machine translation model. Please help in translating the following text to {target_lang}."
    _MT_TASK_PROMPT = (
        f"Translate the following English text to {target_lang}. Follow these guidelines:\n\n"
        "1. **Accuracy:** Translate the text as accurately as possible, ensuring no information is missed.\n"
        "2. **Tone and Naturalness:** Maintain the original tone and ensure the translation sounds natural.\n"
        "3. **Meaningful Restructuring:** Do not translate word-for-word. Restructure sentences if needed.\n"
        "4. **Do not Translate the numericals\n"
        "5. **Do not provide any explanations.\n\n"
        "Ensure that the translated text is clear, coherent, and grammatically correct.\n\n"
    )

    # Translate the input text
    translated_prompt = translate_text(llama, _MT_SYSTEM_PROMPT, _MT_TASK_PROMPT, row['sentence_eng_Latn'])

    output_dir = f"/content/IndicTrans2/{lang}"
    os.makedirs(output_dir, exist_ok=True)
    output_data = {
        'source_lang': 'eng_Latn',
        'target_lang': _FLORES_MAP[_LANG_MAP[lang]],
        'meta': {
            'model_name': 'Meta-Llama-3.1-405B-Instruct-FP32',
            'temperature': _TEMPERATURE,
            'max_tokens': _MAX_TOKENS,
            'top_p': _TOP_P,
            'frequency_penalty': _FREQUENCY_PENALTY,
            'presence_penalty': _PRESENCE_PENALTY
        },
        'Input Prompt': row['sentence_eng_Latn'],
        'translated_text': translated_prompt,
        'id': int(row['id']),
    }

    # Save the result to a JSONL file
    with open(f"{output_dir}/{lang}_translations_{i}.jsonl", 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False)

# Main execution
if __name__ == "__main__":
    # Load the dataset
    in22_dataset = load_dataset("ai4bharat/IN22-Conv", "eng_Latn-asm_Beng")
    df = pd.DataFrame(in22_dataset["conv"])
    lang = "asm"  # Set your target language code here

    # Prepare arguments for parallel processing
    tasks = [(i, row, lang) for i, row in df.iterrows()]

    # Set the number of jobs (adjust based on your environment)
    num_jobs = 5  # You can set this to 5 or 10 as per your requirement

    # Run translations in parallel
    Parallel(n_jobs=num_jobs)(
        delayed(process_translation)(args) for args in tqdm(tasks)
    )
'''

In [ ]:
#from google.colab import files
#files.download("/content/in22-translations")

In [ ]:
#in22_dataset = load_dataset("ai4bharat/IN22-Conv","eng_Latn-asm_Beng")


In [ ]:
#in22_dataset = load_dataset("ai4bharat/IN22-Conv")

In [ ]:
#df.iloc[0]

In [ ]:
#target_lang = "Bengali"
#_MT_SYSTEM_PROMPT = f"You are a machine translation model. Please help in translating the following text to {target_lang}."
#_MT_TASK_PROMPT = (
 '''   f"Translate the following English text to {target_lang}. Follow these guidelines:\n\n"
    "1. **Accuracy:** Translate the text as accurately as possible, ensuring no information is missed.\n"
    "2. **Tone and Naturalness:** Maintain the original tone and ensure the translation sounds natural.\n"
    "3. **Meaningful Restructuring:** Do not translate word-for-word. Restructure sentences if needed.\n"
    "4. **Do not Translate the numericals\n"
    "5. **Do not provide any Explnations.\n\n"
    "Ensure that the translated text is clear, coherent, and grammatically correct.\n\n"
)
llama = OpenAI(
        base_url = "https://integrate.api.nvidia.com/v1",
        api_key = "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"
    )
# Translate Prompt and Answer
translated_prompt = translate_text(llama, _MT_SYSTEM_PROMPT, _MT_TASK_PROMPT, df.iloc[10]['sentence_eng_Latn'])'''

In [ ]:
###translated_prompt

In [ ]:
#in22_dataset["conv"][0]

In [ ]:
#! lscpu